In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
import seaborn as sns
from sequencing_tools.viz_tools import okabeito_palette, color_encoder, simpsons_palette
from sequencing_tools.stats_tools import p_adjust
from scipy.special import ndtr
from collections import defaultdict
from sequencing_tools.fastq_tools import reverse_complement
from sequencing_tools.bam_tools import get_strand
import RNA
from multiprocessing import Pool
import random
import pysam
import glob
import re
#import genomeview
from pybedtools import BedTool
import mappy as mp
from plotting_utils import figure_path
from matplotlib import rcParams
from peak_utils import *
from tblout_parser import read_tbl

plt.rc('axes', labelsize=15)
plt.rc('xtick', labelsize = 15)
plt.rc('ytick', labelsize = 15)
rcParams['font.family'] = 'sans-serif'
rcParams['font.sans-serif'] = ['Arial']

In [2]:
project_path = '/stor/work/Lambowitz/cdw2854/cfNA/tgirt_map'
peak_path = project_path + '/bed_files/merged_bed/MACS2/annotated'
#peak_path = project_path + '/CLAM//BED_files/peaks/annotation'
peak_tsv = peak_path + '/unfragmented.tsv'
print(peak_tsv)
peak_df = load_peaks(peak_tsv)  \
    .assign(sense_gtype = lambda d: np.where(d.sense_gtype == ".", 'Unannotated', d.sense_gtype))\
    .assign(antisense_gtype = lambda d: np.where(d.antisense_gtype == ".", 'Unannotated', d.antisense_gtype)) \
    .sort_values('pileup', ascending=False)
peak_df.head()

/stor/work/Lambowitz/cdw2854/cfNA/tgirt_map/bed_files/merged_bed/MACS2/annotated/unfragmented.tsv


,chrom,start,end,peakname,score,fc,log10p,log10q,pileup,sample_count,sense_gname,sense_gtype,strand,antisense_gname,antisense_gtype,pvalue,FDR,is_sense
46743,chr16,223472,223709,unfragmented.rvs_peak_9830,22989,593.97974,2306.09131,2298.91455,775.0,15,.,Unannotated,-,LSM11,RBP,0.0,0.0,Antisense
46744,chr16,230338,230554,unfragmented.rvs_peak_9833,19003,478.41898,1907.42175,1900.30774,690.0,12,.,Unannotated,-,NCBP2,RBP,0.0,0.0,Antisense
46742,chr10,71355030,71355103,unfragmented.rvs_peak_3389,28895,674.90723,2897.06421,2889.54199,684.0,14,.,Unannotated,-,.,Unannotated,0.0,0.0,Unannotated
46745,chr16,227336,227447,unfragmented.rvs_peak_9832,18507,478.65143,1857.76416,1850.75354,661.0,12,.,Unannotated,-,YBX3,RBP,0.0,0.0,Antisense
0,chr11,65273309,65273587,unfragmented.fwd_peak_5292,21282,509.75793,2135.26367,2128.24292,516.0,15,GTF2F1,RBP,+,TALAM1,Long RNA,0.0,0.0,Sense


In [16]:
anti_peaks = peak_df.query('is_sense == "Antisense"')\
    .pipe(lambda d: d[~d.antisense_gtype.str.contains('tRF')])\
    .query('pileup >= %i & sample_count > %i' %(pileup_cutoff, sample_cutoff)) \
    .assign(seq = lambda d: list(map(fetch_seq,d.chrom, d.start, d.end, d.strand)))
anti_peaks.to_csv(peak_path + '/antisense_peaks.csv',index=False)
anti_peaks.head()

,chrom,start,end,peakname,score,fc,log10p,log10q,pileup,sample_count,sense_gname,sense_gtype,strand,antisense_gname,antisense_gtype,pvalue,FDR,is_sense,seq
46743,chr16,223472,223709,unfragmented.rvs_peak_9830,22989,593.97974,2306.09131,2298.91455,775.0,15,.,Unannotated,-,LSM11,RBP,0.0,0.0,Antisense,TGCTGCCCACTCAGACTTTATTCAAAGACCAGGAAGGGCCGGTGCA...
46744,chr16,230338,230554,unfragmented.rvs_peak_9833,19003,478.41898,1907.42175,1900.30774,690.0,12,.,Unannotated,-,NCBP2,RBP,0.0,0.0,Antisense,GCCGACGTTGCTGCCCAGCTTCTTCCACAGGGCGCGCACCAGCGCC...
46745,chr16,227336,227447,unfragmented.rvs_peak_9832,18507,478.65143,1857.76416,1850.75354,661.0,12,.,Unannotated,-,YBX3,RBP,0.0,0.0,Antisense,CCCAAGGGGCAAGAAGCATGGCCACCGAGGCTCCAGCTTAACGGTA...
46746,chr16,230708,230855,unfragmented.rvs_peak_9834,7634,233.06796,769.93719,763.45087,252.0,12,.,Unannotated,-,YBX3,RBP,0.0,0.0,Antisense,GGTCCACTCGCAGCTGGCACGCGTGCAGGTGGCTCAGCGCGGACAG...
46750,chr11,5275567,5275689,unfragmented.fwd_peak_4635,3946,130.41985,400.80322,394.60443,175.0,12,.,Unannotated,+,SND1,RBP,0.0,0.0,Antisense,CTCAGCTGGGCAAAGGTGCCCTTGAGATCATCCAGGTGCTTTATGG...


In [19]:
with open(peak_path + '/antisense_peaks.fa', 'w') as fa:
    for i, row in anti_peaks.iterrows():
        fa.write('>%s\n%s\n' %(row['peakname'], row['seq']))